<a href="https://colab.research.google.com/github/BNIA/VitalSigns/blob/main/CitiStat_Lights_Create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://bniajfi.org/indicators/Crime%20and%20Safety/lights

- Topic Area:   Crime and Safety

- Source:   Baltimore City CitiStat

- Years Available:   2016, 2017, 2018

- The rate of service requests for addressing street light outages made through Baltimore's 311 system per 1,000 residents. More than one service request may be made for the same issue but is logged as a unique request.

Number 215 on the indicator Sheet

## SETUP Enviornment:

### Import Modules

In [ ]:
%%capture
! pip install -U -q PyDrive
! pip install geopy
! pip install geopandas
! pip install geoplot
! pip install dataplay
! pip install matplotlib
! pip install psycopg2-binary

In [ ]:
%%capture
! apt-get install build-dep python-psycopg2
! apt-get install libpq-dev
! apt-get install libspatialindex-dev

In [ ]:
%%capture
!pip install rtree
!pip install dexplot

In [ ]:
from dataplay.geoms import workWithGeometryData

In [ ]:
%%capture 
# These imports will handle everything
import os
import sys
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import psycopg2
import pyproj
from pyproj import Proj, transform
# conda install -c conda-forge proj4
from shapely.geometry import Point
from shapely import wkb
from shapely.wkt import loads
# https://pypi.org/project/geopy/
from geopy.geocoders import Nominatim

# In case file is KML, enable support
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Configure Enviornment

In [ ]:
# This will just beautify the output

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.precision', 2)
# pd.reset_option('max_colwidth')
pd.set_option('max_colwidth', 20)
# pd.reset_option('max_colwidth')

## Prep Datasets

#### TPOP CSA and Baltimore

Get Baltimore

In [ ]:
#collapse_output
#collapse_input
csa = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/0/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"
csa = gpd.read_file(csa);
csa.head(1)

Get CSA

In [ ]:
url2 = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/1/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"
csa2 = gpd.read_file(url2);
csa2['CSA2010'] = csa2['City_1'] 
csa2['OBJECTID'] = 56 
csa2 = csa2.drop(columns=['City_1'])
csa2.head()

Append do no append Bcity. We put it on the Bottom of the df because when performing the ponp it returns only the last matching columns CSA Label. 

In [ ]:
# csa = pd.concat([csa2, csa], ignore_index=True)
csa = csa.append(csa2).reset_index(drop=True)

In [ ]:
csa.head(3)

In [ ]:
csa.tail(3)

In [ ]:
csa.head()

In [ ]:
csa.drop(columns=['Shape__Area', 'Shape__Length', 'OBJECTID'], axis=1).to_file("BCity_and_CSA.geojson", driver='GeoJSON')

### Citistat

In [ ]:
ls

In [ ]:
lights = gpd.read_file("LightsOut_2019_CSACity.shp");

In [ ]:
year = '20'

In [ ]:
original = gpd.read_file("LightsOut_20"+year+"_CSACity.shp");

In [ ]:
original.rename(columns={ 'CSA':'CSA2010', 'BaltCity':'InBaltimore'}, inplace=True)

In [ ]:
original.head()

In [ ]:
# Convert to EPSG:4326
# fares.crs
original = original.to_crs(epsg=4326)

Original Dataset

In [ ]:
original.plot()

Remove these for not being either a CSA or Baltimore

In [ ]:
removeThese = original[ original['CSA2010'].isnull() & original['InBaltimore'].isnull()  ]
removeThese.plot()

Keep These

In [ ]:
df = original[ original['CSA2010'].notnull() | original['InBaltimore'].notnull()  ]
df.plot()

In [ ]:
print('After filtering records where a CSA or Baltimore geo-code match Exists')
print( 'All rows Before Filter: ', original.shape[0] ) # rows, columns
print( '# w BCity.isnull: ', df.InBaltimore.isnull().sum() ); bmorow = df[ df.CSA2010.isnull()  ].shape[0]
print( '# w CSA2010.isnull: ', bmorow ); csarow = df[ df.CSA2010.notnull()  ].shape[0] 
print( '# w CSA2010.notnull: ', csarow ); 
print( '# rows After Filter: ', df.shape[0],'==',csarow,'+',bmorow,'==', csarow + bmorow); 

In [ ]:
# add baltimore city
df.CSA2010 = df.CSA2010.fillna('Baltimore City')

In [ ]:
df.head(1)

In [ ]:
df.to_csv('citistat_ponp_gdf'+year+'.csv', index=False) 

In [ ]:
# list(fares.columns)
print(df.columns.values)

### Create Indicator - 215 - Lights

The rate of service requests for addressing street light outages made through Baltimore's 311 system per 1,000 residents.

More than one service request may be made for the same issue but is logged as a unique request.

In [ ]:
lightsCsa = df.copy()

In [ ]:
citistat.dtypes

In [ ]:
lightsCsa['count'] = 1
lightsCsa.groupby('CSA2010').sum(numeric_only=True).head(56)

In [ ]:
csa.tail()

In [ ]:
  lightsCsaTemp = df.copy()
  lightsCsaTemp['count'] = 1
  light = lightsCsaTemp.groupby('CSA2010').sum(numeric_only=True) 
  df.head(1)

In [ ]:
light['count'].sum()

In [ ]:
def lightsOut(df,yr):
  # The rate of service requests for addressing street light outages made through Baltimore's 311 system per 1,000 residents.
  # More than one service request may be made for the same issue but is logged as a unique request.

  # Create the Denominator
  lightDenominator = csa.copy()
  lightDenominator = lightDenominator[['tpop10','CSA2010']]
  lightDenominator = lightDenominator.set_index('CSA2010')

  # Create the Numerator
  light = df.copy()
  light['count'] = 1
  light = light.groupby('CSA2010').sum(numeric_only=True) 

  # Make sure ALL csas and BaltimoreCity are included and sorted.
  light = csa.merge( light, left_on='CSA2010', right_on='CSA2010', how='outer' )  
  light.drop( columns=['geometry', 'Shape__Length','CouncilDis','Latitude','Longitude','Shape__Area','OBJECTID_y','OBJECTID_x'], inplace=True)
  light.at[55,'count']=light['count'].sum()
  # Perform the calculation
  light['lights_'+year] = light['count'] / light['tpop10'] * 1000 

  light.to_csv('215-lights'+yr+'.csv', index=False)

  print( 'Records Matching Query: ', light.size / len(light.columns) )
  return light

lightsOut(lightsCsa,'_'+year).tail(5)